In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import os
import shutil
directory_out = './file/3.part'
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

import html as html_lib

# = = =  = = =  = = =

import json

# = = =  = = =  = = =

directory_in = './file/2.url'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                           header=0,
                                                                           dtype=str).fillna('')
    print('Loading Done !')
    print()

    df_input['No'] = df_input['No'].astype(int)

    crawler_remain = len(df_input)

    print(f'总数量：{crawler_remain}')
    print()

    # = = =  = = =  = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, crawler_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = =  = = =  = = =

        try:
            crawler_retry = 0
            while True:
                crawler_retry += 1

                try:
                    url_request = crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0].strip()
                        if type_:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = =  = = =  = = =

            dict_src, i, list_href = {}, 0, list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
            if list_href:
                for href in list_href:
                    if href.startswith('http'):
                        dict_src[str(i)] = href
                        i += 1
            else:
                list_href = [href.strip() for href in html.xpath('//a[@id="ProductPic"]/@href')]
                dict_src = {'0': list_href[0]} if list_href else {}

            # = = =  = = =  = = =

            part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()

            # = = =  = = =  = = =

            description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()

            # = = =  = = =  = = =

            list_text = [text.strip() for text in html.xpath('//div[@id="divAppSummary"]/p/text()')]
            while '' in list_text:
                list_text.remove('')

            application_summary = ';'.join(list_text)

            # = = =  = = =  = = =

            list_div = html.xpath('//div[@class="alert alert-warning"]')
            list_text = [etree.tostring(div).decode('utf-8') for div in list_div]
            list_text = [re.sub(r'<.*?>', '', text) for text in list_text]
            list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
            list_text = [html_lib.unescape(text) for text in list_text]

            if len(list_text) == 1:
                warning = list_text[0]
            else:
                warning = ''
                for i, text in enumerate(list_text):
                    warning += f'{i + 1}. {text}\n'

            # = = =  = = =  = = =

            brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()

            # = = =  = = =  = = =

            value = html.xpath('//input[@id="hidfDetailApp"]/@value')
            vehicle_url = f'https://www.dormanproducts.com/{value[0].strip()}' if value else ''

            # = = =  = = =  = = =

            dict_key, list_tr = {}, html.xpath('//section[@id="productSpec"]/div/table/tr')
            for tr in list_tr:
                key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
                if key in dict_key:
                    dict_key[key] += 1
                else:
                    dict_key[key] = 1

            dict_specification, dict_count = {}, {}
            for i, tr in enumerate(list_tr):
                key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
                if dict_key[key] == 1:
                    dict_specification[str(i)] = {key: tr.xpath('./td/text()')[0].strip()}
                else:
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                        dict_count[key] = 1
                    dict_specification[str(i)] = {f'{key}-{dict_count[key]}': tr.xpath('./td/text()')[0].strip()}

            # = = =  = = =  = = =

            list_text = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
            oem = ';'.join(list(dict.fromkeys(list_text)))

            # = = =  = = =  = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Type': type_,
                                          'Part Number': part_number,
                                          'Brand': brand,
                                          'Description': description,
                                          'Warning': warning.strip(),
                                          'Application Summary': application_summary,
                                          'OEM': oem,
                                          'Picture': '',
                                          'Url': crawler_series['Url'],
                                          'Json_Src': json.dumps(dict_src),
                                          'Vehicle Url': vehicle_url,
                                          'Json_Specification': json.dumps(dict_specification)})

            # = = =  = = =  = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = =  = = =  = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and crawler_remain == 1 or crawler_remain % 30 == 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            crawler_remain -= 1

    # = = =  = = =  = = =

    pool.map(crawler, df_input.iterrows())

    print('Data Outputting...')
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Type', 'Part Number'],
                                                                 ascending=[True, True],
                                                                 ignore_index=True)
        df_correct['No'] = range(1, len(df_correct) + 1)
        df_correct.to_excel(os.path.join(directory_out, f'''{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
    print('Output Finished !')
    print()
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('- Error Occurred -')
        print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

[True] - [请求1次] - [剩余29条] - [1（100.00%）] - [2024/12/13 09:42:38] - 154. https://www.dormanproducts.com/p-100922-963-545d.aspx?origin=keyword

[True] - [请求3次] - [剩余28条] - [1（100.00%）] - [2024/12/13 09:42:38] - 45. https://www.dormanproducts.com/p-133607-635-698.aspx?origin=keyword

[True] - [请求2次] - [剩余27条] - [1（100.00%）] - [2024/12/13 09:42:38] - 131. https://www.dormanproducts.com/p-84478-635-549.aspx?origin=keyword

[True] - [请求2次] - [剩余26条] - [1（100.00%）] - [2024/12/13 09:42:38] - 134. https://www.dormanproducts.com/p-153694-696-428.aspx?origin=keyword

[True] - [请求2次] - [剩余25条] - [1（100.00%）] - [2024/12/13 09:42:39] - 127. https://www.dormanproducts.com/p-76420-917-759.aspx?origin=keyword

[True] - [请求1次] - [剩余24条] - [1（100.00%）] - [2024/12/13 09:42:39] - 156. https://www.dormanproducts.com/p-81060-030-502.aspx?origin=keyword

[True] - [请求3次] - [剩余23条] - [1（100.00%）] - [2024/12/13 09:42:39] - 130. https://www.dormanproducts.com/p-86349-902-5933.aspx?origin=keyword

[True] - [请求1次] 